# SpaceX Launch Records Dashboard

This code creates an interactive dashboard to visualize SpaceX launch data. Users can explore launch success rates and correlations between payload mass and launch success.

**Functionalities:**

* **Launch Site Selection:** Choose a launch site from a dropdown menu (all sites are included by default).
* **Success Rate Visualization:** View a pie chart that shows the success rate for all launch sites or a specific launch site if selected.
* **Payload Mass Range Selection:** Use a slider to choose a range for payload mass.
* **Scatter Plot:** View a scatter chart that shows the correlation between payload mass and launch success, colored by booster version category.

In [ ]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',
                                    options=[
                                        {'label': 'All Sites', 'value': 'ALL'},
                                        {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                        {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                        {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                        {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}
                                    ],
                                    value='ALL',
                                    placeholder="Insert Launch Site",
                                    searchable=True
                                    ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                                min=0, max=10000, step=1000,
                                                marks = {i: str(i) for i in range(0, 10001, 1000)},
                                                value=[0, 10000]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart'))
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
# Function decorator to specify function input and output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        df = spacex_df['Launch Site'].value_counts().reset_index()
        fig = px.pie(df, values='count', 
                names='Launch Site', 
                title='Launch Sites Success Rate')
        return fig
    else:
        # return the outcomes piechart for a selected site
        df = spacex_df[spacex_df['Launch Site'] == entered_site]
        df = df['class'].value_counts().reset_index()
        fig = px.pie(df, values='count', 
                names= df['class'].map({0:'Failure', 1:'Success'}).values, 
                title= entered_site + ' Launch Site Success Rate')
        return fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'),
              Input(component_id='payload-slider', component_property='value')])
def get_scatter_plot(entered_site, entered_payload):
    minVal, maxVal = entered_payload[0], entered_payload[1]
    if entered_site == 'ALL':
        df = spacex_df[(spacex_df['Payload Mass (kg)'] >= minVal) & (spacex_df['Payload Mass (kg)'] <= maxVal)]
        fig = px.scatter(df,
                x='Payload Mass (kg)', 
                y='class', 
                color='Booster Version Category')
        return fig
    else:
        # return the outcomes scatterplot for a selected site
        df = spacex_df[spacex_df['Launch Site'] == entered_site]
        df = df[(df['Payload Mass (kg)'] >= minVal) & (spacex_df['Payload Mass (kg)'] <= maxVal)]
        fig = px.scatter(df,
                x='Payload Mass (kg)', 
                y='class', 
                color='Booster Version Category')
        return fig

# Run the app
if __name__ == '__main__':
    app.run_server()

## Project Completed by:
[Karim Nasr](https://www.linkedin.com/in/karim-nasr-abu-al-fath/)